In [1]:
import os
import csv
import pandas as pd

In [ ]:
#Download the CA & AZ HMDA Dataset

if not os.path.exists("../input_data/state_AZ-CA_mini.csv"):
    hmda = pd.read_csv('../../FIN 388/FinTech-Capstone-2023/input_data/state_AZ-CA.csv',low_memory=False,header=0)
    hmda.sample(50000).to_csv("../../FIN 388/FinTech-Capstone-2023/input_data/state_AZ-CA_mini.csv",index=False)


In [2]:
#Viewing the dataset
miniCA_AZ = pd.read_csv("../../FIN 388/FinTech-Capstone-2023/input_data/state_AZ-CA_mini.csv")
miniCA_AZ

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2021,549300ZJIC4LOLZH0M42,40900,CA,6067.0,6.067010e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,6578,91.23,90900,70,1084,1840,31
1,2021,549300DD5QQUHO6PCH70,40140,CA,6065.0,6.065041e+09,NC,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,7928,81.71,77500,193,1386,1713,0
2,2021,549300HW662MN1WU8550,41740,CA,6073.0,6.073009e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,5055,59.07,95100,93,985,998,46
3,2021,5493005FJKWE0GR4YS94,40900,CA,6017.0,6.017032e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,2941,23.70,90900,236,826,903,13
4,2021,549300FGXN1K3HLB1R50,33700,CA,6099.0,6.099001e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,NaN,NaN,NaN,18628,47.41,68900,159,4043,5181,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2021,54930021WPEXNHYZUL09,44700,CA,6077.0,6.077004e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,3359,64.69,74000,46,517,1019,48
49996,2021,549300FGXN1K3HLB1R50,36084,CA,6013.0,6.013329e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6735,49.37,125600,84,1587,2143,56
49997,2021,549300FX7K8PTEQUU487,31084,CA,6037.0,6.037504e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,9.0,NaN,NaN,5284,57.48,80000,146,1154,1427,58
49998,2021,549300ALNLUNS3Y53T24,38060,AZ,4013.0,4.013061e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,6807,31.22,79000,129,1553,2302,11


In [3]:
import cenpy
import segregation
import contextily
%matplotlib inline

ImportError: Geopandas is required to do spatial operations, and must be installed in order to use the cenpy product API. For directions on how to install geopandas, consult https://geopandas.org/install.html. Ensure that all of the dependencies, including rtree, are installed. Normally, installing geopandas through the  Anaconda Software Distribution (https://repo.continuum.io) in the "conda-forge" software channel will work.

In [ ]:
#Downloading ACS data
acs = cenpy.products.ACS(2019)
acs.variables

acs.variables.to_csv("../../FIN 388/FinTech-Capstone-2023/input_data/2019cenpyvariablenames.csv")
acs.tables.to_csv("../../FIN 388/FinTech-Capstone-2023/input_data/2019cenpyvariablenamestable.csv")

In [ ]:
az = acs.from_state('AZ')
az

In [ ]:
az = acs.from_state('AZ', variables =['B19001A_001E','B01003_001E'])
# az.shape # of tracts

## 'B02001_001E'is the RACE table for variable, 
    # but it counts for the total number of people recorded for race,
    # therefore, it has the same number from variable 'B01003_001E'recording total population in the region

#renaming the census variable for merging purposes
az.rename(columns={'GEOID':'census_tract',
                   'B19001A_001E':'Total Household Income',
                   'B01003_001E':'Total Population'}, inplace=True)
az

In [ ]:
az = acs.from_state('AZ',level='tract',variables=['B01002H_001E'])
# az.shape # of tracts

# #renaming the census variable for merging purposes
# az.rename(columns={'GEOID':'census_tract'}, inplace=True)

az

In [ ]:
# Making both census tract codes match by removing the leading '0' from ACS dataset
az["census_tract"] = int(az["census_tract"])
az

In [ ]:
# Making both census tract codes match by removing the leading '0' from ACS dataset
az["census_tract"] = az["census_tract"].astype(str)
az["census_tract"] = az["census_tract"].str.lstrip('0')
az

In [ ]:
# Converting the variable back to an int (it was a string above)
az["census_tract"] = az["census_tract"].astype(int)

#merging the dataset
merge = az.merge(df1, on = "census_tract", how = "inner")

#View the merged dataset
merge.head()